# Metaclasses in Python

---

## Table of Contents
1. What are Metaclasses?
2. type() - The Default Metaclass
3. Creating Classes Dynamically
4. Custom Metaclasses
5. Metaclass Methods
6. Metaclass Inheritance
7. Practical Use Cases
8. Metaclasses vs Decorators
9. Key Points
10. Practice Exercises

---

## 1. What are Metaclasses?

**Everything in Python is an object**, including classes.

- Objects are instances of classes
- Classes are instances of metaclasses
- The default metaclass is `type`

```
instance -> class -> metaclass -> type
  obj    ->  Foo  ->   type   -> type
```

**Metaclasses control class creation behavior.**

In [ ]:
# Everything is an object
class MyClass:
    pass

obj = MyClass()

print(f"obj is instance of MyClass: {isinstance(obj, MyClass)}")
print(f"MyClass is instance of type: {isinstance(MyClass, type)}")
print(f"type is instance of type: {isinstance(type, type)}")

In [ ]:
# Class hierarchy
print(f"Type of obj: {type(obj)}")
print(f"Type of MyClass: {type(MyClass)}")
print(f"Type of type: {type(type)}")

In [ ]:
# Even built-in types follow this pattern
print(f"Type of int: {type(int)}")
print(f"Type of str: {type(str)}")
print(f"Type of list: {type(list)}")

---

## 2. type() - The Default Metaclass

In [ ]:
# type() has two uses:
# 1. Get the type of an object
print(type(42))      # <class 'int'>
print(type("hello")) # <class 'str'>

# 2. Create a new class dynamically
# type(name, bases, dict)

In [ ]:
# These are equivalent:

# Method 1: class statement
class Dog:
    species = "Canis familiaris"
    
    def __init__(self, name):
        self.name = name
    
    def bark(self):
        return f"{self.name} says woof!"

# Method 2: type() function
def init(self, name):
    self.name = name

def bark(self):
    return f"{self.name} says woof!"

DogType = type(
    'DogType',                     # Class name
    (),                            # Base classes (tuple)
    {                              # Class dictionary
        'species': 'Canis familiaris',
        '__init__': init,
        'bark': bark
    }
)

# Both work the same
d1 = Dog("Buddy")
d2 = DogType("Max")

print(d1.bark())
print(d2.bark())
print(f"Same species: {d1.species == d2.species}")

In [ ]:
# Creating class with inheritance
class Animal:
    def breathe(self):
        return "Breathing..."

# Create Cat class inheriting from Animal
Cat = type(
    'Cat',
    (Animal,),  # Inherits from Animal
    {
        'sound': 'meow',
        'make_sound': lambda self: f"The cat says {self.sound}"
    }
)

cat = Cat()
print(cat.breathe())    # From Animal
print(cat.make_sound()) # From Cat

---

## 3. Creating Classes Dynamically

In [ ]:
# Factory function to create classes
def create_class(class_name, attributes):
    """Create a class with given attributes."""
    return type(class_name, (), attributes)

# Create different classes dynamically
Point2D = create_class('Point2D', {
    '__init__': lambda self, x, y: setattr(self, '_coords', (x, y)) or None,
    'x': property(lambda self: self._coords[0]),
    'y': property(lambda self: self._coords[1]),
    '__repr__': lambda self: f"Point2D({self.x}, {self.y})"
})

Point3D = create_class('Point3D', {
    '__init__': lambda self, x, y, z: setattr(self, '_coords', (x, y, z)) or None,
    'x': property(lambda self: self._coords[0]),
    'y': property(lambda self: self._coords[1]),
    'z': property(lambda self: self._coords[2]),
    '__repr__': lambda self: f"Point3D({self.x}, {self.y}, {self.z})"
})

p2 = Point2D(3, 4)
p3 = Point3D(1, 2, 3)
print(p2)
print(p3)

In [ ]:
# Create enum-like classes dynamically
def create_enum(name, values):
    """Create an enum-like class from a list of values."""
    attrs = {v: v for v in values}
    attrs['_values'] = tuple(values)
    attrs['__iter__'] = lambda self: iter(self._values)
    return type(name, (), attrs)

Color = create_enum('Color', ['RED', 'GREEN', 'BLUE'])
Status = create_enum('Status', ['PENDING', 'ACTIVE', 'COMPLETED'])

print(f"Color.RED = {Color.RED}")
print(f"Status values: {Status._values}")

In [ ]:
# Create data class dynamically
def create_data_class(name, fields):
    """Create a simple data class with given fields."""
    def init(self, **kwargs):
        for field in fields:
            setattr(self, field, kwargs.get(field))
    
    def repr_method(self):
        values = ', '.join(f"{f}={getattr(self, f)!r}" for f in fields)
        return f"{name}({values})"
    
    def eq_method(self, other):
        if not isinstance(other, self.__class__):
            return False
        return all(getattr(self, f) == getattr(other, f) for f in fields)
    
    return type(name, (), {
        '_fields': fields,
        '__init__': init,
        '__repr__': repr_method,
        '__eq__': eq_method
    })

Person = create_data_class('Person', ['name', 'age', 'email'])

p1 = Person(name='Alice', age=30, email='alice@example.com')
p2 = Person(name='Alice', age=30, email='alice@example.com')

print(p1)
print(f"p1 == p2: {p1 == p2}")

---

## 4. Custom Metaclasses

In [ ]:
# Creating a metaclass by subclassing type
class SimpleMeta(type):
    """A simple metaclass that prints when a class is created."""
    
    def __new__(mcs, name, bases, namespace):
        print(f"Creating class: {name}")
        print(f"  Bases: {bases}")
        print(f"  Attributes: {list(namespace.keys())}")
        
        # Create the class
        cls = super().__new__(mcs, name, bases, namespace)
        return cls

# Use the metaclass
class MyClass(metaclass=SimpleMeta):
    x = 10
    
    def method(self):
        pass

In [ ]:
# Metaclass that adds attributes
class AutoAttrMeta(type):
    """Metaclass that adds automatic attributes."""
    
    def __new__(mcs, name, bases, namespace):
        # Add a created_at attribute
        import datetime
        namespace['_created_at'] = datetime.datetime.now()
        namespace['_class_name'] = name
        
        return super().__new__(mcs, name, bases, namespace)

class TrackedClass(metaclass=AutoAttrMeta):
    pass

print(f"Class name: {TrackedClass._class_name}")
print(f"Created at: {TrackedClass._created_at}")

In [ ]:
# Metaclass that enforces rules
class AbstractMeta(type):
    """Metaclass that enforces abstract methods."""
    
    def __new__(mcs, name, bases, namespace):
        cls = super().__new__(mcs, name, bases, namespace)
        
        # Check for abstract methods from bases
        abstract_methods = set()
        for base in bases:
            if hasattr(base, '_abstract_methods'):
                abstract_methods.update(base._abstract_methods)
        
        # Check if all abstract methods are implemented
        for method in abstract_methods:
            if method not in namespace or getattr(namespace[method], '_is_abstract', False):
                raise TypeError(
                    f"Can't instantiate class {name} with abstract method {method}"
                )
        
        return cls

def abstract_method(func):
    func._is_abstract = True
    return func

class BaseClass(metaclass=AbstractMeta):
    _abstract_methods = {'do_something'}
    
    @abstract_method
    def do_something(self):
        pass

# This will work
class ConcreteClass(BaseClass):
    def do_something(self):
        return "Implemented!"

c = ConcreteClass()
print(c.do_something())

---

## 5. Metaclass Methods

In [ ]:
# __new__ vs __init__ in metaclasses
class VerboseMeta(type):
    def __new__(mcs, name, bases, namespace):
        print(f"__new__ called: Creating class {name}")
        # __new__ creates and returns the class
        cls = super().__new__(mcs, name, bases, namespace)
        return cls
    
    def __init__(cls, name, bases, namespace):
        print(f"__init__ called: Initializing class {name}")
        # __init__ is called after __new__
        super().__init__(name, bases, namespace)

class Example(metaclass=VerboseMeta):
    pass

In [ ]:
# __call__ in metaclasses - controls instance creation
class SingletonMeta(type):
    """Metaclass that makes classes singletons."""
    _instances = {}
    
    def __call__(cls, *args, **kwargs):
        # __call__ is invoked when we do ClassName()
        if cls not in cls._instances:
            print(f"Creating new instance of {cls.__name__}")
            instance = super().__call__(*args, **kwargs)
            cls._instances[cls] = instance
        else:
            print(f"Returning existing instance of {cls.__name__}")
        return cls._instances[cls]

class Database(metaclass=SingletonMeta):
    def __init__(self):
        self.connection = "Connected"

db1 = Database()
db2 = Database()
print(f"db1 is db2: {db1 is db2}")

In [ ]:
# __prepare__ - customize the namespace dict
from collections import OrderedDict

class OrderedMeta(type):
    """Metaclass that preserves attribute definition order."""
    
    @classmethod
    def __prepare__(mcs, name, bases):
        # Return custom namespace dict (called before class body executes)
        return OrderedDict()
    
    def __new__(mcs, name, bases, namespace):
        # namespace is the OrderedDict with attributes in definition order
        cls = super().__new__(mcs, name, bases, dict(namespace))
        cls._field_order = list(namespace.keys())
        return cls

class OrderedFields(metaclass=OrderedMeta):
    first = 1
    second = 2
    third = 3

print(f"Field order: {OrderedFields._field_order}")

In [ ]:
# __getattribute__ and __setattr__ in metaclasses
class ReadOnlyMeta(type):
    """Metaclass that makes class attributes read-only."""
    
    def __setattr__(cls, name, value):
        if name.startswith('_'):
            # Allow setting private attributes
            super().__setattr__(name, value)
        else:
            raise AttributeError(f"Cannot modify read-only attribute: {name}")

class Config(metaclass=ReadOnlyMeta):
    DEBUG = False
    VERSION = "1.0"
    _internal = "can be changed"

print(f"DEBUG: {Config.DEBUG}")
print(f"VERSION: {Config.VERSION}")

try:
    Config.DEBUG = True  # Will raise error
except AttributeError as e:
    print(f"Error: {e}")

# Private attributes can be changed
Config._internal = "modified"
print(f"_internal: {Config._internal}")

---

## 6. Metaclass Inheritance

In [ ]:
# Metaclasses are inherited by subclasses
class LoggingMeta(type):
    def __new__(mcs, name, bases, namespace):
        print(f"LoggingMeta: Creating {name}")
        return super().__new__(mcs, name, bases, namespace)

class BaseLogged(metaclass=LoggingMeta):
    pass

# Subclass inherits the metaclass
class DerivedLogged(BaseLogged):
    pass

print(f"Type of BaseLogged: {type(BaseLogged)}")
print(f"Type of DerivedLogged: {type(DerivedLogged)}")

In [ ]:
# Metaclass conflict
class MetaA(type):
    pass

class MetaB(type):
    pass

class ClassA(metaclass=MetaA):
    pass

class ClassB(metaclass=MetaB):
    pass

# This would cause a conflict:
# class ClassC(ClassA, ClassB):  # TypeError!
#     pass

# Solution: Create a combined metaclass
class MetaC(MetaA, MetaB):
    pass

class ClassC(ClassA, ClassB, metaclass=MetaC):
    pass

print(f"Type of ClassC: {type(ClassC)}")

In [ ]:
# Metaclass with multiple inheritance
class ValidationMeta(type):
    def __new__(mcs, name, bases, namespace):
        print(f"ValidationMeta processing {name}")
        return super().__new__(mcs, name, bases, namespace)

class RegistryMeta(type):
    registry = []
    
    def __new__(mcs, name, bases, namespace):
        cls = super().__new__(mcs, name, bases, namespace)
        mcs.registry.append(cls)
        print(f"RegistryMeta registered {name}")
        return cls

# Combined metaclass
class CombinedMeta(ValidationMeta, RegistryMeta):
    def __new__(mcs, name, bases, namespace):
        # Calls both parent metaclasses
        return super().__new__(mcs, name, bases, namespace)

class MyModel(metaclass=CombinedMeta):
    pass

print(f"Registry: {[c.__name__ for c in CombinedMeta.registry]}")

---

## 7. Practical Use Cases

In [ ]:
# Use Case 1: Plugin System / Registry
class PluginMeta(type):
    plugins = {}
    
    def __new__(mcs, name, bases, namespace):
        cls = super().__new__(mcs, name, bases, namespace)
        # Don't register the base class
        if bases:  # Only register subclasses
            plugin_name = namespace.get('name', name.lower())
            mcs.plugins[plugin_name] = cls
        return cls
    
    @classmethod
    def get_plugin(mcs, name):
        return mcs.plugins.get(name)

class Plugin(metaclass=PluginMeta):
    """Base plugin class."""
    def execute(self):
        raise NotImplementedError

class JSONPlugin(Plugin):
    name = 'json'
    def execute(self):
        return "Processing JSON"

class XMLPlugin(Plugin):
    name = 'xml'
    def execute(self):
        return "Processing XML"

# Use the registry
print(f"Available plugins: {list(PluginMeta.plugins.keys())}")

# Get and use a plugin
json_plugin = PluginMeta.get_plugin('json')()
print(json_plugin.execute())

In [ ]:
# Use Case 2: ORM-style Field Definition
class Field:
    def __init__(self, field_type, required=False):
        self.field_type = field_type
        self.required = required
        self.name = None
    
    def __repr__(self):
        return f"Field({self.field_type.__name__}, required={self.required})"

class ModelMeta(type):
    def __new__(mcs, name, bases, namespace):
        # Collect all Field instances
        fields = {}
        for key, value in namespace.items():
            if isinstance(value, Field):
                value.name = key
                fields[key] = value
        
        namespace['_fields'] = fields
        return super().__new__(mcs, name, bases, namespace)

class Model(metaclass=ModelMeta):
    def __init__(self, **kwargs):
        for name, field in self._fields.items():
            value = kwargs.get(name)
            if field.required and value is None:
                raise ValueError(f"{name} is required")
            if value is not None and not isinstance(value, field.field_type):
                raise TypeError(f"{name} must be {field.field_type.__name__}")
            setattr(self, name, value)

class User(Model):
    name = Field(str, required=True)
    age = Field(int)
    email = Field(str, required=True)

# Test the model
print(f"User fields: {User._fields}")

user = User(name="Alice", age=30, email="alice@example.com")
print(f"User: {user.name}, {user.age}, {user.email}")

try:
    User(name="Bob")  # Missing required email
except ValueError as e:
    print(f"Error: {e}")

In [ ]:
# Use Case 3: API Versioning
class APIVersionMeta(type):
    versions = {}
    
    def __new__(mcs, name, bases, namespace):
        cls = super().__new__(mcs, name, bases, namespace)
        
        version = namespace.get('version')
        if version:
            api_name = name.replace('V' + str(version), '')
            if api_name not in mcs.versions:
                mcs.versions[api_name] = {}
            mcs.versions[api_name][version] = cls
        
        return cls
    
    @classmethod
    def get_api(mcs, name, version=None):
        apis = mcs.versions.get(name, {})
        if version:
            return apis.get(version)
        # Return latest version
        if apis:
            return apis[max(apis.keys())]
        return None

class API(metaclass=APIVersionMeta):
    pass

class UserAPIV1(API):
    version = 1
    def get_users(self):
        return "V1: Basic user list"

class UserAPIV2(API):
    version = 2
    def get_users(self):
        return "V2: Enhanced user list with pagination"

# Get specific version
v1 = APIVersionMeta.get_api('UserAPI', 1)()
print(v1.get_users())

# Get latest version
latest = APIVersionMeta.get_api('UserAPI')()
print(latest.get_users())

In [ ]:
# Use Case 4: Automatic Method Decoration
import functools
import time

def timed(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        print(f"{func.__name__} took {time.time() - start:.4f}s")
        return result
    return wrapper

class TimedMeta(type):
    """Metaclass that adds timing to all methods."""
    
    def __new__(mcs, name, bases, namespace):
        # Decorate all callable attributes
        for attr_name, attr_value in namespace.items():
            if callable(attr_value) and not attr_name.startswith('_'):
                namespace[attr_name] = timed(attr_value)
        
        return super().__new__(mcs, name, bases, namespace)

class SlowOperations(metaclass=TimedMeta):
    def operation1(self):
        time.sleep(0.1)
        return "op1"
    
    def operation2(self):
        time.sleep(0.2)
        return "op2"

ops = SlowOperations()
ops.operation1()
ops.operation2()

In [ ]:
# Use Case 5: Enforce Interface
class InterfaceMeta(type):
    """Metaclass that enforces interface implementation."""
    
    def __new__(mcs, name, bases, namespace):
        cls = super().__new__(mcs, name, bases, namespace)
        
        # Get required methods from _interface attribute
        required = getattr(cls, '_interface', [])
        
        # Check implementation
        for method_name in required:
            method = getattr(cls, method_name, None)
            if method is None or getattr(method, '_not_implemented', False):
                raise TypeError(
                    f"{name} must implement {method_name}()"
                )
        
        return cls

def not_implemented(func):
    func._not_implemented = True
    return func

class Serializable(metaclass=InterfaceMeta):
    _interface = ['serialize', 'deserialize']
    
    @not_implemented
    def serialize(self):
        pass
    
    @not_implemented
    def deserialize(self, data):
        pass

# This works - implements both methods
class JSONSerializable(Serializable):
    def serialize(self):
        return "JSON data"
    
    def deserialize(self, data):
        return "Parsed data"

js = JSONSerializable()
print(js.serialize())

---

## 8. Metaclasses vs Decorators

In [ ]:
# Many things can be done with both
# Here's a comparison:

# Using a class decorator
def add_greeting(cls):
    cls.greet = lambda self: f"Hello from {cls.__name__}"
    return cls

@add_greeting
class WithDecorator:
    pass

# Using a metaclass
class GreetingMeta(type):
    def __new__(mcs, name, bases, namespace):
        namespace['greet'] = lambda self: f"Hello from {name}"
        return super().__new__(mcs, name, bases, namespace)

class WithMetaclass(metaclass=GreetingMeta):
    pass

# Both work
print(WithDecorator().greet())
print(WithMetaclass().greet())

In [ ]:
# When to use which:
comparison = """
| Feature                    | Decorator | Metaclass |
|----------------------------|-----------|----------|
| Modify single class        | Good      | Overkill |
| Affect all subclasses      | Manual    | Automatic|
| Control class creation     | Limited   | Full     |
| Combine multiple behaviors | Stack     | Inherit  |
| Simplicity                 | Simple    | Complex  |
| Explicit application       | Yes       | Inherited|

Use decorators when:
- Modifying a single class
- Simple modifications
- Want explicit application

Use metaclasses when:
- Need to affect all subclasses automatically
- Need to control class creation process
- Building frameworks/libraries
- Need __prepare__ or similar hooks
"""
print(comparison)

In [ ]:
# Decorators don't affect subclasses automatically
@add_greeting
class Parent:
    pass

class Child(Parent):
    pass

print(f"Parent has greet: {hasattr(Parent, 'greet')}")
# Child inherits greet from Parent, but if we want
# the decorator logic applied, we'd need to decorate Child too

# Metaclasses DO affect subclasses
class MetaParent(metaclass=GreetingMeta):
    pass

class MetaChild(MetaParent):
    pass

print(f"MetaChild has greet: {hasattr(MetaChild, 'greet')}")

---

## 9. Key Points

1. **Classes are objects**: Created by metaclasses (default is `type`)
2. **type()**: Can be used to create classes dynamically
3. **Custom metaclass**: Subclass `type` and override methods
4. **__new__**: Called to create the class object
5. **__init__**: Called to initialize the class object
6. **__call__**: Called when creating instances (e.g., for singletons)
7. **__prepare__**: Returns the namespace dict before class body executes
8. **Inheritance**: Metaclasses are inherited by subclasses
9. **Use cases**: Registries, ORMs, singletons, validation, interfaces
10. **Alternative**: Often class decorators are simpler and sufficient

---

## 10. Practice Exercises

In [ ]:
# Exercise 1: Create a metaclass that automatically adds a __repr__ method
# The __repr__ should show all instance attributes

class AutoReprMeta(type):
    pass

# Test:
# class Point(metaclass=AutoReprMeta):
#     def __init__(self, x, y):
#         self.x = x
#         self.y = y
# 
# p = Point(3, 4)
# print(p)  # Should show: Point(x=3, y=4)

In [ ]:
# Exercise 2: Create a metaclass that validates class attributes
# - All public attributes must have type annotations
# - All public methods must have docstrings

class StrictMeta(type):
    pass

# Test:
# class Good(metaclass=StrictMeta):
#     value: int = 10
#     
#     def method(self):
#         """This method does something."""
#         pass
#
# class Bad(metaclass=StrictMeta):  # Should raise error
#     value = 10  # No type annotation

In [ ]:
# Exercise 3: Create a metaclass for a simple command pattern
# - Each class method becomes a registered command
# - Commands can be executed by name

class CommandMeta(type):
    pass

class CommandHandler(metaclass=CommandMeta):
    pass

# Test:
# class Calculator(CommandHandler):
#     def add(self, a, b):
#         return a + b
#     
#     def subtract(self, a, b):
#         return a - b
#
# calc = Calculator()
# print(calc.execute('add', 5, 3))  # 8

In [ ]:
# Exercise 4: Create a metaclass that implements observer pattern
# - Track all attribute changes
# - Notify observers when attributes change

class ObservableMeta(type):
    pass

# Test:
# class Person(metaclass=ObservableMeta):
#     def __init__(self, name):
#         self.name = name
#
# def observer(obj, attr, old, new):
#     print(f"{attr} changed from {old} to {new}")
#
# p = Person("Alice")
# p.add_observer(observer)
# p.name = "Bob"  # Should trigger observer

In [ ]:
# Exercise 5: Create a metaclass for immutable classes
# - After __init__, no attributes can be modified
# - Raise error on attribute modification attempt

class ImmutableMeta(type):
    pass

# Test:
# class Point(metaclass=ImmutableMeta):
#     def __init__(self, x, y):
#         self.x = x
#         self.y = y
#
# p = Point(3, 4)
# print(p.x)  # 3
# p.x = 5  # Should raise error

---

## Solutions

In [ ]:
# Solution 1:
class AutoReprMeta(type):
    def __new__(mcs, name, bases, namespace):
        def auto_repr(self):
            attrs = ', '.join(f"{k}={v!r}" for k, v in self.__dict__.items())
            return f"{name}({attrs})"
        
        namespace['__repr__'] = auto_repr
        return super().__new__(mcs, name, bases, namespace)

class Point(metaclass=AutoReprMeta):
    def __init__(self, x, y):
        self.x = x
        self.y = y

p = Point(3, 4)
print(p)

In [ ]:
# Solution 2:
class StrictMeta(type):
    def __new__(mcs, name, bases, namespace):
        annotations = namespace.get('__annotations__', {})
        
        for attr_name, attr_value in namespace.items():
            if attr_name.startswith('_'):
                continue
            
            # Check for type annotations on attributes
            if not callable(attr_value):
                if attr_name not in annotations:
                    raise TypeError(f"Attribute '{attr_name}' must have type annotation")
            
            # Check for docstrings on methods
            if callable(attr_value) and not attr_value.__doc__:
                raise TypeError(f"Method '{attr_name}' must have a docstring")
        
        return super().__new__(mcs, name, bases, namespace)

class Good(metaclass=StrictMeta):
    value: int = 10
    
    def method(self):
        """This method does something."""
        pass

print("Good class created successfully")

try:
    class Bad(metaclass=StrictMeta):
        value = 10  # No type annotation
except TypeError as e:
    print(f"Error: {e}")

In [ ]:
# Solution 3:
class CommandMeta(type):
    def __new__(mcs, name, bases, namespace):
        commands = {}
        
        for attr_name, attr_value in namespace.items():
            if callable(attr_value) and not attr_name.startswith('_'):
                commands[attr_name] = attr_value
        
        namespace['_commands'] = commands
        
        def execute(self, command_name, *args, **kwargs):
            if command_name not in self._commands:
                raise ValueError(f"Unknown command: {command_name}")
            return self._commands[command_name](self, *args, **kwargs)
        
        namespace['execute'] = execute
        return super().__new__(mcs, name, bases, namespace)

class Calculator(metaclass=CommandMeta):
    def add(self, a, b):
        return a + b
    
    def subtract(self, a, b):
        return a - b

calc = Calculator()
print(f"add: {calc.execute('add', 5, 3)}")
print(f"subtract: {calc.execute('subtract', 10, 4)}")
print(f"Available commands: {list(calc._commands.keys())}")

In [ ]:
# Solution 4:
class ObservableMeta(type):
    def __new__(mcs, name, bases, namespace):
        original_init = namespace.get('__init__', lambda self: None)
        
        def new_init(self, *args, **kwargs):
            self._observers = []
            original_init(self, *args, **kwargs)
        
        def add_observer(self, observer):
            self._observers.append(observer)
        
        def __setattr__(self, name, value):
            if name != '_observers' and hasattr(self, '_observers'):
                old_value = getattr(self, name, None)
                object.__setattr__(self, name, value)
                for observer in self._observers:
                    observer(self, name, old_value, value)
            else:
                object.__setattr__(self, name, value)
        
        namespace['__init__'] = new_init
        namespace['add_observer'] = add_observer
        namespace['__setattr__'] = __setattr__
        
        return super().__new__(mcs, name, bases, namespace)

class Person(metaclass=ObservableMeta):
    def __init__(self, name):
        self.name = name

def observer(obj, attr, old, new):
    print(f"{attr} changed from {old!r} to {new!r}")

p = Person("Alice")
p.add_observer(observer)
p.name = "Bob"
p.name = "Charlie"

In [ ]:
# Solution 5:
class ImmutableMeta(type):
    def __new__(mcs, name, bases, namespace):
        original_init = namespace.get('__init__', lambda self: None)
        
        def new_init(self, *args, **kwargs):
            object.__setattr__(self, '_initialized', False)
            original_init(self, *args, **kwargs)
            object.__setattr__(self, '_initialized', True)
        
        def __setattr__(self, name, value):
            if getattr(self, '_initialized', False):
                raise AttributeError(
                    f"Cannot modify attribute '{name}' of immutable object"
                )
            object.__setattr__(self, name, value)
        
        def __delattr__(self, name):
            raise AttributeError(
                f"Cannot delete attribute '{name}' of immutable object"
            )
        
        namespace['__init__'] = new_init
        namespace['__setattr__'] = __setattr__
        namespace['__delattr__'] = __delattr__
        
        return super().__new__(mcs, name, bases, namespace)

class Point(metaclass=ImmutableMeta):
    def __init__(self, x, y):
        self.x = x
        self.y = y

p = Point(3, 4)
print(f"Point: ({p.x}, {p.y})")

try:
    p.x = 5
except AttributeError as e:
    print(f"Error: {e}")